In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import os
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
sys.path.append('/home/jma819/post_cmfe_analysis')
#sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import dlc_utils

In [3]:
from importlib import reload
reload(dlc_utils)

<module 'dlc_utils' from '/home/jma819/post_cmfe_analysis/dlc_utils.py'>

In [4]:
#dlc_directory = '/volumes/My_Passport/dlc_analysis/behavcamvideos/open_field_miniscope/'
dlc_directory = '/projects/b1118/behaviorvideos/'

In [5]:
experiment_folders = [folder for folder in os.listdir(dlc_directory) if '_H' in folder]

In [6]:
experiment_folders

['GRIN013_H13_M33_S54']

In [7]:
for folder in tqdm(experiment_folders):
    output_directory = dlc_directory+folder+'/'
    session_name = folder
    print(folder)
    dlc_output_files = os.listdir(output_directory)
    dlc_output_files.sort()
    
    names = []
    dlc_dfs = []
    for output_file in dlc_output_files:
        if output_file.endswith('000.h5'):
            print(output_file)
            dlc_output = pd.read_hdf(output_directory + output_file)
            dlc_output = dlc_utils.calculate_centroid(dlc_output)
            dlc_output.head()
            coordinates_delta_df = dlc_utils.difference_df(dlc_output)
            df_columns = list(dlc_output.columns)
            print('calculating velocity')
            velocity_df = pd.DataFrame(np.transpose(np.array([np.array([dlc_utils.velocity(coordinates_delta_df[body_part]['x'].values[frame],coordinates_delta_df[body_part]['y'].values[frame]) for frame in range(len(coordinates_delta_df))]) for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))]))])), 
                                       columns=list(set([df_columns[item][1] for item in range(len(df_columns))]))) 
            for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))])):
                velocity_df[body_part+'_likelihood']=dlc_output[df_columns[0][0]][body_part]['likelihood'].values
            
            names.append(output_file)
            dlc_dfs.append(velocity_df)

    session_compiled = pd.concat(dlc_dfs, keys=names)

    #save compiled session 
    session_compiled.to_hdf(output_directory+session_name+'_compiled_session_02112021.h5', key='df')

    #create behavior data aligned to the msCam frames
    aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_session_02112021.h5', output_directory+'timestamp.dat')
    aligned.reset_index(inplace=True)
    aligned['msCamFrame'] = aligned.index.values
    aligned = aligned.set_index(pd.to_timedelta(np.linspace(0, len(aligned)*(1/20), len(aligned)), unit='s'), drop=False)

    #smooth data by downsampling and interpolating 
    interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')
    interpolated.to_csv(output_directory+session_name+'_dlc_tracking_foranalysis_02112021.csv')

    output_directory+session_name+'_dlc_tracking_foranalysis.csv'

  0%|          | 0/1 [00:00<?, ?it/s]

GRIN013_H13_M33_S54
behavCam01DLC_resnet50_Open_Field_v2Jan27shuffle1_150000.h5


AttributeError: module 'pandas' has no attribute 'read_hdf'

In [ ]:
#fixing errors
GRIN012_H17_M46_S35_error
GRIN033_H13_M42_S33
GRIN033_H14_M34_S32_01162020

In [ ]:
#aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5', output_directory+'timestamp.dat')
dlc_analysis = pd.read_hdf(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5')

In [ ]:
frame_clock_df = pd.read_table(output_directory+'timestamp.dat')

In [ ]:
dlc_full = dlc_full.reset_index()

In [ ]:
msCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0].set_index('frameNum')
behavCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1].set_index('frameNum')
msCam_timestamps['sysClock'][1] = 0
behavCam_timestamps['sysClock'][1] = 0

In [ ]:
msCam_timestamps

In [ ]:
behavCam_timestamps

In [ ]:
msCam_timestamps = dlc_utils.align_behavior_data(msCam_timestamps, behavCam_timestamps)
msCam_timestamps.reset_index(inplace=True)